In [77]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
import time

In [50]:
train_data = torchvision.datasets.MNIST(root = "./data", download = True, train = True, transform = torchvision.transforms.ToTensor())
test_data = torchvision.datasets.MNIST(root = "./data", download = True, train = False, transform = torchvision.transforms.ToTensor())

In [47]:
train_data_batches = DataLoader(train_data, batch_size = 8, shuffle = True)
test_data_batches = DataLoader(test_data, batch_size = 8, shuffle = True)

In [127]:
class MNISTClassifier(nn.Module):
    def __init__(self):
        super(MNISTClassifier, self).__init__()
        self.layer1 = nn.Linear(28 * 28, 392)
        self.bc1 = nn.BatchNorm1d(392)
        self.dropout = nn.Dropout(0.5)
        self.layer2 = nn.Linear(392, 128)
        self.bc2 = nn.BatchNorm1d(128)
        self.layer3 = nn.Linear(128, 10)
    def forward(self, x):
        x = x.view(-1, 28 * 28)
        
        x = self.layer1(x)
        x = self.bc1(x)
        x = F.relu(x)
        # x = self.dropout(x)
        
        x = self.layer2(x)
        x = self.bc2(x)
        x = F.relu(x)
        # x = self.dropout(x)
        
        x = self.layer3(x)
        return x


In [128]:
t1 = time.time()
model = MNISTClassifier()
print(model.parameters())

loss_fn = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

epochs = 10

for epoch in range(epochs):
    for batch_x, batch_y in train_data_batches:
        y = model(batch_x)
        optimizer.zero_grad()
        loss = loss_fn(y, batch_y)
        loss.backward()
        optimizer.step()
    scheduler.step()
    print(f"epoch: {epoch}, loss: {loss}")

print("Time Taken:", time.time() - t1)

<generator object Module.parameters at 0x72debd54fa00>
epoch: 0, loss: 0.08451491594314575
epoch: 1, loss: 0.0033451940398663282
epoch: 2, loss: 0.29710081219673157
epoch: 3, loss: 0.036811575293540955
epoch: 4, loss: 0.8894553780555725
epoch: 5, loss: 0.04256376251578331
epoch: 6, loss: 0.06969285756349564
epoch: 7, loss: 0.027622731402516365
epoch: 8, loss: 0.01625262014567852
epoch: 9, loss: 0.000624795735348016
Time Taken: 170.801913022995


In [129]:
from matplotlib import pyplot as plt
import numpy as np
def show_batch(batch):
    im = torchvision.utils.make_grid(batch)
    plt.imshow(np.transpose(im.numpy(), (1, 2, 0)))

def calculate_accuracy(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_x, batch_y in test_data_batches:
            # images = images.view(images.size(0), -1)
            y = model(batch_x)
            _, pred_val = torch.max(y, 1)
            # show_batch(batch_x)

            total += batch_y.size(0)  # Total number of labels
            correct += (pred_val == batch_y).sum().item()
            
    accuracy = 100 * correct / total
    return accuracy

t1 = time.time()
accuracy_perc = calculate_accuracy(model, test_data_batches)
print(accuracy_perc, time.time() - t1)

96.44 1.135613203048706


In [ ]:
# Accuracy Normal - 97.44, time: 84 sec approx
# Accuracy Batch Normalisation: 96.12
# Accuracy Dropout: 97.18
# Accuracy Batch and Dropout: 94.59
# Accuracy Dropout and Batch: 94.96
#MAX: 97.83, 97.76